In [111]:
from zipfile import ZipFile
import pandas as pd
import kaggle
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [112]:
# https://www.kaggle.com/kimjihoo/coronavirusdataset
kaggle.api.authenticate()
kg_url='kimjihoo/coronavirusdataset'
files=['patient.csv','route.csv','time.csv']

In [113]:
#kaggle.api.dataset_download_files('sudalairajkumar/novel-corona-virus-2019-dataset')

In [114]:
#for file in files:
#    kaggle.api.dataset_download_file(kg_url,file,force=True, path='data')#force =Trure rewrites file if excist

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Mar 2020 19:24:12 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ka_sessionid=4c47dec5141aa5d218278c62c4438af4; max-age=15759359; path=/, GCLB=CImr4KLcu6XHYA; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/datasets/download/kimjihoo/coronavirusdataset/patient.csv', 'X-Kaggle-MillisecondsElapsed': '194', 'X-Kaggle-RequestId': 'abec610c70254708ef4ddd2d4b012f95', 'X-Kaggle-ApiVersion': '1.5.6', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-e9PfNFYvLijLWji0wJxVwQ==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://poly

In [ ]:
df_pat = pd.read_csv('data/'+files[0])
df_route = pd.read_csv('data/'+files[1])
df_time = pd.read_csv('data/'+files[2])
#df_time =df_time.reset_index()

In [ ]:
df_patient = pd.read_csv('data/patient.csv')

In [ ]:
df_patient.info()

In [ ]:
#df_time.rename(columns={'176':'Date'},inplace=True)
df_time['date']=pd.to_datetime(df_time['date'])

In [ ]:
df_patient.info()

In [ ]:
date_clm = ['confirmed_date','released_date','deceased_date']
for clm in date_clm:
    df_patient[clm]=pd.to_datetime(df_patient[clm])

In [ ]:
df_patient['duration']=df_patient['released_date']-df_patient['confirmed_date']
df_patient['duration_death']=df_patient['deceased_date']-df_patient['confirmed_date']
df_patient['age'] =2019- df_patient['birth_year']

In [ ]:
df_patient['age'].hist(bins=20)
plt.xlabel('age')

In [ ]:
df_patient['duration'].astype('timedelta64[D]').plot.hist(label='Recovered')
df_patient['duration_death'].astype('timedelta64[D]').plot.hist(label='Deseased')
plt.xlabel('days')
plt.legend()
plt.title('Duration of disease')
plt.show()

In [ ]:
df_patient[df_patient['state']=='deceased']['age'].hist(bins=20,alpha=0.5,label='decesed')
df_patient[df_patient['state']=='released']['age'].hist(bins=20,alpha=0.5,label='released')
plt.xlabel('age')
plt.title('Patient age distribution')
plt.legend();

In [ ]:
df_time=df_time.set_index('date')


In [ ]:
df_time['test'].plot(label='Tested')
df_time['confirmed'].plot(label='Confirmed')
df_time['released'].plot(label='Released')
plt.legend()


In [ ]:
import plotly.express as px

In [ ]:
df_time.head()

In [ ]:
fig = px.line(title='Cases')
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['confirmed'], mode='lines',name='Confiermed')
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['released'], mode='lines', name="Released")
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['deceased'], mode='lines', name="Deceased")
fig.show()

In [ ]:
fig = px.histogram(df_patient, x='age', range_x=[0,100], range_y=[0,30])
fig.show()

In [ ]:
df_patient.info()

## Model


In [151]:
df_p =pd.read_csv('data/patient.csv')
df_p.drop(columns=['patient_id','country','region',
                  'infected_by','infection_reason',
                   'infection_order','contact_number','group',
                  'disease'],inplace=True)
date_list =['confirmed_date','released_date','deceased_date']
for name in date_list:
    df_p[name]=pd.to_datetime(df_p[name])
df_p['sex'].fillna('Unkown', inplace=True)

In [152]:
df_p['recovery_length']=df_p['released_date']-df_p['confirmed_date']
df_p['before_deceased_days']=df_p['deceased_date']-df_p['confirmed_date']

In [153]:
df_p.head()

,sex,birth_year,confirmed_date,released_date,deceased_date,state,recovery_length,before_deceased_days
0,female,1984.0,2020-01-20,2020-02-06,NaT,released,17 days,NaT
1,male,1964.0,2020-01-24,2020-02-05,NaT,released,12 days,NaT
2,male,1966.0,2020-01-26,2020-02-12,NaT,released,17 days,NaT
3,male,1964.0,2020-01-27,2020-02-09,NaT,released,13 days,NaT
4,male,1987.0,2020-01-30,2020-03-02,NaT,released,32 days,NaT


In [163]:
df_for_model=df_p[['sex','birth_year','state','recovery_length','before_deceased_days']]
df_for_model.shape


(7869, 5)

In [184]:
df_for_model = df_for_model[df_for_model['birth_year'].notna()]
df_for_model.fillna( pd.Timedelta(seconds=0), inplace=True)
df_for_model['recovery_length']=df_for_model['recovery_length'].dt.days.astype('int16')
df_for_model['before_deceased_days']=df_for_model['before_deceased_days'].dt.days.astype('int16')

In [211]:
df_for_model=df_for_model[(df_for_model['state']=='deceased') | (df_for_model['state']=='released' )]
df_for_model.birth_year  = df_for_model.birth_year.astype('int16')


In [212]:
df_for_model

,sex,birth_year,state,recovery_length,before_deceased_days
0,female,1984,released,17,0
1,male,1964,released,12,0
2,male,1966,released,17,0
3,male,1964,released,13,0
4,male,1987,released,32,0
...,...,...,...,...,...
5459,male,1940,deceased,0,1
5766,male,1938,deceased,0,0
5767,female,1935,deceased,0,0
6284,female,1929,deceased,0,-2


In [213]:

df_hot_encoded = pd.get_dummies(df_for_model, drop_first=True) 

In [214]:
X = df_hot_encoded.drop('state_released',axis=1)
Y = df_hot_encoded['state_released']

In [215]:
from sklearn.model_selection import train_test_split as tts

In [216]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2)

In [205]:
from imblearn.over_sampling import SMOTE

In [229]:
sm = SMOTE(sampling_strategy='minority', random_state=7)
oversampled_trainX, oversampled_trainY = sm.fit_sample(X_train, y_train)


In [230]:
from xgboost import XGBClassifier

In [231]:
xgb= XGBClassifier()

In [234]:
xgb.fit(oversampled_trainX,oversampled_trainY)
xgb.score(X_test, y_test)

1.0

In [235]:
xgb.fit(X_train,y_train)
xgb.score(X_test, y_test)

1.0